In [1]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

# model, dl_train, dl_eval = load_robust_experiment("Standard_R50", "imagenet")
model, dl_train, dl_eval = load_torchvision_experiment("googlenet", batch_size=256)

In [ ]:
from ulib.attack import StopCriteria
from ulib.attacks.svrg_uap import SVRG_UAP
from ulib.pert_module import PertModule
from ulib import eval

stop = StopCriteria(max_epochs=15, max_time=600)
pert_model = PertModule(
    model,
    data_shape=dl_train.get_tensor(0).shape[1:],
    eps=10 / 255,
    random_init=False,
)

criterion = torch.nn.CrossEntropyLoss()
learning_rate = 0.5
num_batches = len(dl_train)

attack = SVRG_UAP(
    pert_model=pert_model,
    criterion=criterion,
    num_batches=num_batches,
    learning_rate=learning_rate,
    y_s_eps=12 / 255,
    attk_correct=True,
    mode="average",
    metric_func=eval.fooling_rate,
)

pert = attack.fit(dl_train, dl_eval, stop)

In [4]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)